In [1]:
from lxml import html, etree

In [2]:

import requests


In [3]:
import pandas as pd

In [4]:
import fastparquet

In [5]:
import numpy as np

In [2]:
page = requests.get('https://www.gov.uk/government/publications/register-of-licensed-sponsors-workers')
tree = html.fromstring(page.content)

In [3]:
download_path = tree.xpath('//*[@id="attachment_4271537"]/div[2]/h2/a')[0].get("href")

In [4]:
print(download_path)

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/891847/2020-06-12_Tier_2_5_Register_of_Sponsors.pdf


In [5]:
response = requests.get(download_path)

In [6]:
with open('/tmp/metadata.pdf', 'wb') as f:
    f.write(response.content)

In [ ]:
# python3 pdf2txt.py metadata.pdf --output_type xml  --outfile raw.xml

In [ ]:
# python3 pdf2txt.py metadata.pdf --output_type html  --outfile raw.html

In [31]:
# with open('./raw-sample.html', 'r') as f:
#     page = f.read()
# doc = html.fromstring(page)

In [86]:
with open('./raw_html_all.html', 'r') as f:
    page = f.read()
doc = html.fromstring(page)

In [104]:
doc.xpath('/html/body/div[7]')[0].text

## Extract Organisation Name

In [245]:
name = []
top_skipped = []

isFirst = False

for col in doc.xpath("//div/span"):
    if ("left:30px;" in col.getparent().get("style")):
        if isFirst or ("top:266px;" in col.getparent().get("style")):
            
            p_style = col.getparent().get("style")
            pairs = {'style': p_style, 'name': col.text.strip()}
            
            p_styles = p_style.split(';')
            for i in range(0, (len(p_styles)-1)):
                key_value = p_styles[i].split(':')
                if key_value[0].strip() in ['top', 'left', 'width', 'height']:
                    pairs[key_value[0].strip()] = int(key_value[1].replace('px','').strip())
#                 else:
#                     pairs[key_value[0].strip()] = key_value[1].replace('px','').strip()
            
#             print(pairs)
            if pairs['top'] in top_skipped:
                print("skipped as already imported once, top = ",pairs['top'])
                continue

            ## handle the city - start
            if (df_city_sorted[ (df_city_sorted['top'] - pairs['top']).abs() < 10 ].shape[0]) == 1:
                pairs['city_top'] = (df_city_sorted[ (df_city_sorted['top'] - pairs['top']).abs() < 10 ].iloc[0]['top'])
                pairs['city'] = (df_city_sorted[ (df_city_sorted['top'] - pairs['top']).abs() < 10 ].iloc[0]['name'])
            else:
                if (len(col.getparent().getchildren())==2):
                    print("more than one sibling", len(col.getparent().getchildren()), pairs['top']);
                    elem_0 = len(col.getparent().getchildren()[0].text.strip())
                    elem_1 = len(col.getparent().getchildren()[1].text.strip())
#                     print(elem_0)
#                     print(elem_1)
                    if(elem_0 > elem_1):
                        pairs['name'] = col.getparent().getchildren()[0].text.strip()
                        pairs['city'] = col.getparent().getchildren()[1].text.strip()
                        pairs['city_top'] = pairs['top']
                    else:
                        pairs['name'] = col.getparent().getchildren()[1].text.strip()
                        pairs['city'] = col.getparent().getchildren()[0].text.strip()
                        pairs['city_top'] = pairs['top']
                        
                    top_skipped.append(pairs['top'])
                    
                elif (len(col.getparent().getchildren())==1):
                    print("one sibling, top=",pairs['top']);
                else:
                    print("sibling : else, top=",pairs['top']);
                
#                 break;
            ## handle the city - end
            try:
                next_top = df_city_sorted[ (df_city_sorted['top'] > pairs['top'] + 10)].iloc[0]['top']
            except:
                next_top = pairs['top'] + 100
    
            ## handle the Tier & rating - start
#             filtered_df_tier_and_rating_sorted = df_tier_and_rating_sorted[ (df_tier_and_rating_sorted['adjected_top'] - pairs['top']).abs() < 10 ]
#             if (filtered_df_tier_and_rating_sorted.shape[0]) == 1:
#                 pairs['tier_and_rating_top'] = (filtered_df_tier_and_rating_sorted.iloc[0]['top'])
#                 pairs['tier_and_rating'] = filtered_df_tier_and_rating_sorted.iloc[0]['name']
#             elif filtered_df_tier_and_rating_sorted.shape[0] > 1:
#                 pairs['tier_and_rating_top'] = filtered_df_tier_and_rating_sorted.iloc[0]['top']
#                 pairs['tier_and_rating'] = "\n".join( filtered_df_tier_and_rating_sorted['name'].tolist() )
            filtered_df_tier_and_rating_sorted = df_tier_and_rating_sorted[ (df_tier_and_rating_sorted['top'] > pairs['top'] ) & (df_tier_and_rating_sorted['top'] < next_top ) ]
            pairs['tier_and_rating_top'] = filtered_df_tier_and_rating_sorted.iloc[0]['top']
            pairs['tier_and_rating'] = "\n".join( filtered_df_tier_and_rating_sorted['name'].tolist() )
            
#             if (df_sub_tier_sorted[ (df_sub_tier_sorted['adjected_top'] - pairs['top']).abs() < 10 ].shape[0]) == 1:
#                 pairs['df_sub_tier_top'] = (df_sub_tier_sorted[ (df_sub_tier_sorted['adjected_top'] - pairs['top']).abs() < 10 ].iloc[0]['top'])
#                 pairs['sub_tier'] = (df_sub_tier_sorted[ (df_sub_tier_sorted['adjected_top'] - pairs['top']).abs() < 10 ].iloc[0]['name'])
            filtered_df_sub_tier_sorted = df_sub_tier_sorted[ (df_sub_tier_sorted['top'] > pairs['top'] ) & (df_sub_tier_sorted['top'] < next_top ) ]
            pairs['sub_tier_top'] = filtered_df_sub_tier_sorted.iloc[0]['top']
            pairs['sub_tier'] = "\n".join( filtered_df_sub_tier_sorted['name'].tolist() )
                    
            name.append(pairs)
            isFirst = True
#             break

df_name = pd.DataFrame(name)
df_name_sorted = df_name.sort_values(by=['top'])
# df_name_sorted[0:]

more than one sibling 2 54489
skipped as already imported once, top =  54489
more than one sibling 2 212611
skipped as already imported once, top =  212611
more than one sibling 2 235838
skipped as already imported once, top =  235838
more than one sibling 2 299158
skipped as already imported once, top =  299158
more than one sibling 2 361484
skipped as already imported once, top =  361484
more than one sibling 2 457767
skipped as already imported once, top =  457767
more than one sibling 2 575206
skipped as already imported once, top =  575206
more than one sibling 2 613200
skipped as already imported once, top =  613200
more than one sibling 2 790679
skipped as already imported once, top =  790679
more than one sibling 2 873101
skipped as already imported once, top =  873101
more than one sibling 2 908309
skipped as already imported once, top =  908309
more than one sibling 2 951463
skipped as already imported once, top =  951463
more than one sibling 2 951493
skipped as already impo

In [243]:
df_name_sorted

,style,name,left,top,width,height,city_top,city,tier_and_rating_top,tier_and_rating,sub_tier_top,sub_tier
0,position:absolute; border: textbox 1px solid; ...,@ Home Accommodation Services Ltd,30,266,160,8,266,London,278,Tier 2 (A rating)\nTier 5 (A rating),279,Tier 2 General\nCreative & Sporting


In [251]:
df_name_sorted[df_name_sorted['top'] == 5755 ]

,style,name,left,top,width,height,city_top,city,tier_and_rating_top,tier_and_rating,sub_tier_top,sub_tier
138,position:absolute; border: textbox 1px solid; ...,66 North UK Limited,30,5755,84,8,5754,London,5893,Tier 2 (A rating)\nTier 2 (A rating),5894,Tier 2 General\nIntra Company Transfers (ICT)


In [247]:
df_name_sorted[df_name_sorted['sub_tier'].isnull() ]

,style,name,left,top,width,height,city_top,city,tier_and_rating_top,tier_and_rating,sub_tier_top,sub_tier


In [248]:
df_tier_and_rating_sorted[ df_tier_and_rating_sorted['top'] == 2967]

,style,name,left,top,width,height,adjected_top
64,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,2967,62,21,2955
65,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,2967,62,21,2955


## Extract Organisation Location

In [161]:
city = []
for col in doc.xpath("//div/span"):
    if ("left:357px;" in col.getparent().get("style")):
        if isFirst or ("top:266px;" in col.getparent().get("style")):
            
            p_style = col.getparent().get("style")
            pairs = {'style': p_style, 'name': col.text.strip()}
            
            p_styles = p_style.split(';')
            for i in range(0, (len(p_styles)-1)):
                key_value = p_styles[i].split(':')
                if key_value[0].strip() in ['top', 'left', 'width', 'height']:
                    pairs[key_value[0].strip()] = int(key_value[1].replace('px','').strip())
#                 else:
#                     pairs[key_value[0].strip()] = key_value[1].replace('px','').strip()
            
#             print(pairs)
#             print(pairs['top'])
            if (df_name_sorted[ (df_name_sorted['top'] - pairs['top']).abs() < 10 ].shape[0]) == 1:
                pairs['name_top'] = (df_name_sorted[ (df_name_sorted['top'] - pairs['top']).abs() < 10 ].iloc[0]['top'])
#             print(pairs)
            city.append(pairs)
            isFirst = True
#             break

In [163]:
df_city = pd.DataFrame(city)

In [165]:
df_city_sorted = df_city.sort_values(by=['top'])

In [169]:
df_city_sorted

,style,name,left,top,width,height,name_top
0,position:absolute; border: textbox 1px solid; ...,London,357,266,29,8,266
1,position:absolute; border: textbox 1px solid; ...,Folkestone,357,309,43,8,309
2,position:absolute; border: textbox 1px solid; ...,London,357,339,29,8,340
3,position:absolute; border: textbox 1px solid; ...,London,357,370,29,8,370
4,position:absolute; border: textbox 1px solid; ...,Slough,357,400,27,8,401
...,...,...,...,...,...,...,...
31275,position:absolute; border: textbox 1px solid; ...,Lichfield,357,1330348,32,8,1330349
31276,position:absolute; border: textbox 1px solid; ...,Blaydon on Tyne,357,1330391,67,8,1330392
31277,position:absolute; border: textbox 1px solid; ...,Wokingham,357,1330422,47,8,1330422
31278,position:absolute; border: textbox 1px solid; ...,London,357,1330465,29,8,1330465


In [167]:
df_city_sorted[df_city_sorted['name_top'].isnull() ]

,style,name,left,top,width,height,name_top


## Extract Tier & Rating

In [149]:
tier_and_rating = []
for col in doc.xpath("//div/span"):
    if ("left:582px;" in col.getparent().get("style")):
        if isFirst or ("top:266px;" in col.getparent().get("style")):
            
            p_style = col.getparent().get("style")
#             print(col.text_content().strip())
#             print(etree.tostring(col))
            pairs = {'style': p_style, 'name': col.text_content().strip()}
            
            p_styles = p_style.split(';')
            for i in range(0, (len(p_styles)-1)):
                key_value = p_styles[i].split(':')
                if key_value[0].strip() in ['top', 'left', 'width', 'height']:
                    pairs[key_value[0].strip()] = int(key_value[1].replace('px','').strip())
#                 else:
#                     pairs[key_value[0].strip()] = key_value[1].replace('px','').strip()
            
#             print(pairs)
            tier_and_rating.append(pairs)
            isFirst = True
#             break

## Extract Sub Tier

In [158]:
sub_tier = []
for col in doc.xpath("//div/span"):
    if ("left:684px;" in col.getparent().get("style")):
        if isFirst or ("top:266px;" in col.getparent().get("style")):
            
            p_style = col.getparent().get("style")
#             print(col.text_content().strip())
#             print(etree.tostring(col))
            pairs = {'style': p_style, 'name': col.text_content().strip()}
            
            p_styles = p_style.split(';')
            for i in range(0, (len(p_styles)-1)):
                key_value = p_styles[i].split(':')
                if key_value[0].strip() in ['top', 'left', 'width', 'height']:
                    pairs[key_value[0].strip()] = int(key_value[1].replace('px','').strip())
#                 else:
#                     pairs[key_value[0].strip()] = key_value[1].replace('px','').strip()
            
#             print(pairs)
            sub_tier.append(pairs)
            isFirst = True
#             break

In [106]:
len(city)

31280

In [99]:
name[101]

{'height': 8,
 'left': 30,
 'name': '3HR PLC',
 'style': 'position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:30px; top:4246px; width:37px; height:8px;',
 'top': 4246,
 'width': 37}

,style,name,left,top,width,height
0,position:absolute; border: textbox 1px solid; ...,@ Home Accommodation Services Ltd,30,266,160,8
1,position:absolute; border: textbox 1px solid; ...,]performance s p a c e [,30,309,98,8
2,position:absolute; border: textbox 1px solid; ...,06472420,30,340,38,8
3,position:absolute; border: textbox 1px solid; ...,0-two Maintenance,30,370,78,8
4,position:absolute; border: textbox 1px solid; ...,1 Digitals Europe Limited,30,401,105,8
...,...,...,...,...,...,...
31314,position:absolute; border: textbox 1px solid; ...,Zytek Automotive Limited,30,1330349,106,8
31315,position:absolute; border: textbox 1px solid; ...,Zytronic Displays Limited,30,1330392,106,8
31316,position:absolute; border: textbox 1px solid; ...,ZyXEL Communications UK Ltd,30,1330422,131,8
31322,position:absolute; border: textbox 1px solid; ...,ZZ&D LtD,30,1330465,40,8


,style,name,left,top,width,height
25,position:absolute; border: textbox 1px solid; ...,Durely,357,1069,25,8
26,position:absolute; border: textbox 1px solid; ...,Newcastle upon Tyne,357,1100,86,8
21,position:absolute; border: textbox 1px solid; ...,Brighton,357,1130,33,8
22,position:absolute; border: textbox 1px solid; ...,LONDON,357,1161,38,8
23,position:absolute; border: textbox 1px solid; ...,London,357,1192,29,8
...,...,...,...,...,...,...
31275,position:absolute; border: textbox 1px solid; ...,Lichfield,357,1330348,32,8
31276,position:absolute; border: textbox 1px solid; ...,Blaydon on Tyne,357,1330391,67,8
31277,position:absolute; border: textbox 1px solid; ...,Wokingham,357,1330422,47,8
31278,position:absolute; border: textbox 1px solid; ...,London,357,1330465,29,8


In [215]:
df_city_sorted[0:]

,style,name,left,top,width,height
0,position:absolute; border: textbox 1px solid; ...,London,357,266,29,8
1,position:absolute; border: textbox 1px solid; ...,Folkestone,357,309,43,8
2,position:absolute; border: textbox 1px solid; ...,London,357,339,29,8
3,position:absolute; border: textbox 1px solid; ...,London,357,370,29,8
4,position:absolute; border: textbox 1px solid; ...,Slough,357,400,27,8
...,...,...,...,...,...,...
31275,position:absolute; border: textbox 1px solid; ...,Lichfield,357,1330348,32,8
31276,position:absolute; border: textbox 1px solid; ...,Blaydon on Tyne,357,1330391,67,8
31277,position:absolute; border: textbox 1px solid; ...,Wokingham,357,1330422,47,8
31278,position:absolute; border: textbox 1px solid; ...,London,357,1330465,29,8


In [105]:
df_name_sorted[25:]

,style,name,left,top,width,height
23,position:absolute; border: textbox 1px solid; ...,1508 London Limited,30,1192,87,8
24,position:absolute; border: textbox 1px solid; ...,15GIFTS LTD,30,1223,54,8
28,position:absolute; border: textbox 1px solid; ...,16HarleyCo Limited,30,1378,81,8
29,position:absolute; border: textbox 1px solid; ...,16printing Ltd,30,1409,58,8
30,position:absolute; border: textbox 1px solid; ...,17 Capital Services Ltd,30,1439,95,8
...,...,...,...,...,...,...
31314,position:absolute; border: textbox 1px solid; ...,Zytek Automotive Limited,30,1330349,106,8
31315,position:absolute; border: textbox 1px solid; ...,Zytronic Displays Limited,30,1330392,106,8
31316,position:absolute; border: textbox 1px solid; ...,ZyXEL Communications UK Ltd,30,1330422,131,8
31322,position:absolute; border: textbox 1px solid; ...,ZZ&D LtD,30,1330465,40,8


In [179]:
df_sub_tier = pd.DataFrame(sub_tier)

In [180]:
df_sub_tier_sorted = df_sub_tier.sort_values(by=['top'])

In [211]:
df_sub_tier_sorted['adjected_top'] = df_sub_tier_sorted['top'] - 12

In [212]:
df_sub_tier_sorted[0:]

,style,name,left,top,width,height,adjected_top
0,position:absolute; border: textbox 1px solid; ...,Tier 2 General\nCreative & Sporting,684,279,77,21,267
1,position:absolute; border: textbox 1px solid; ...,Creative & Sporting,684,322,77,8,310
2,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,353,57,8,341
3,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,383,57,8,371
4,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,414,57,8,402
...,...,...,...,...,...,...,...
32583,position:absolute; border: textbox 1px solid; ...,Tier 2 General\nIntra Company Transfers (ICT),684,1330362,121,21,1330350
32584,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,1330405,57,8,1330393
32585,position:absolute; border: textbox 1px solid; ...,Intra Company Transfers (ICT)\nTier 2 General,684,1330435,121,21,1330423
32586,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,1330478,57,8,1330466


In [159]:
df_tier_and_rating = pd.DataFrame(tier_and_rating)

In [213]:
df_tier_and_rating_sorted = df_tier_and_rating.sort_values(by=['top'])
df_tier_and_rating_sorted['adjected_top'] = df_tier_and_rating_sorted['top'] - 12

In [214]:
df_tier_and_rating_sorted

,style,name,left,top,width,height,adjected_top
0,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating)\nTier 5 (A rating),582,278,62,21,266
1,position:absolute; border: textbox 1px solid; ...,Tier 5 (A rating),582,321,62,8,309
2,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,351,62,8,339
3,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,382,62,8,370
4,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,412,62,8,400
...,...,...,...,...,...,...,...
32673,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating)\nTier 2 (A rating),582,1330360,62,21,1330348
32674,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,1330403,62,8,1330391
32675,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating)\nTier 2 (A rating),582,1330434,62,21,1330422
32676,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,1330477,62,8,1330465


In [7]:
# for content in contents:
#     text_object = content.getObject()
#     print(text_object)

In [8]:
# with open("raw_text.csv", "w") as text_file:
#     text_file.write(all_text)

NameError: name 'all_text' is not defined

In [ ]:
# contents = pageObj.getContents()

## Export dataframe to parquet

In [ ]:
# pip install fastparquet ( if not already installed )

In [258]:
df_name_sorted.to_parquet('df_name_sorted.parquet.gzip',
              compression='gzip')  

In [259]:
df_name_sorted.to_csv('df_name_sorted.csv.gzip',
              compression='gzip')  

In [255]:
df_city_sorted.to_parquet('df_city_sorted.parquet.gzip',
              compression='gzip')  

In [256]:
df_tier_and_rating_sorted.to_parquet('df_tier_and_rating_sorted.parquet.gzip',
              compression='gzip')  

In [257]:
df_sub_tier_sorted.to_parquet('df_sub_tier_sorted.parquet.gzip',
              compression='gzip')  

In [183]:
df_sub_tier_sorted


,style,name,left,top,width,height
0,position:absolute; border: textbox 1px solid; ...,Tier 2 General\nCreative & Sporting,684,279,77,21
1,position:absolute; border: textbox 1px solid; ...,Creative & Sporting,684,322,77,8
2,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,353,57,8
3,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,383,57,8
4,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,414,57,8
...,...,...,...,...,...,...
32583,position:absolute; border: textbox 1px solid; ...,Tier 2 General\nIntra Company Transfers (ICT),684,1330362,121,21
32584,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,1330405,57,8
32585,position:absolute; border: textbox 1px solid; ...,Intra Company Transfers (ICT)\nTier 2 General,684,1330435,121,21
32586,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,1330478,57,8


## Import dataframe from parquet

In [8]:
df_name_sorted_imported = pd.read_parquet('df_name_sorted.parquet.gzip')

In [9]:
df_name_sorted_imported[0:]

,style,name,left,top,width,height,city_top,city,tier_and_rating_top,tier_and_rating,sub_tier_top,sub_tier
index,,,,,,,,,,,,
0,position:absolute; border: textbox 1px solid; ...,@ Home Accommodation Services Ltd,30,266,160,8,266,London,278,Tier 2 (A rating)\nTier 5 (A rating),279,Tier 2 General\nCreative & Sporting
1,position:absolute; border: textbox 1px solid; ...,]performance s p a c e [,30,309,98,8,309,Folkestone,321,Tier 5 (A rating),322,Creative & Sporting
2,position:absolute; border: textbox 1px solid; ...,06472420,30,340,38,8,339,London,351,Tier 2 (A rating),353,Tier 2 General
3,position:absolute; border: textbox 1px solid; ...,0-two Maintenance,30,370,78,8,370,London,382,Tier 2 (A rating),383,Tier 2 General
4,position:absolute; border: textbox 1px solid; ...,1 Digitals Europe Limited,30,401,105,8,400,Slough,412,Tier 2 (A rating),414,Tier 2 General
...,...,...,...,...,...,...,...,...,...,...,...,...
31292,position:absolute; border: textbox 1px solid; ...,Zytek Automotive Limited,30,1330349,106,8,1330348,Lichfield,1330360,Tier 2 (A rating)\nTier 2 (A rating),1330362,Tier 2 General\nIntra Company Transfers (ICT)
31293,position:absolute; border: textbox 1px solid; ...,Zytronic Displays Limited,30,1330392,106,8,1330391,Blaydon on Tyne,1330403,Tier 2 (A rating),1330405,Tier 2 General
31294,position:absolute; border: textbox 1px solid; ...,ZyXEL Communications UK Ltd,30,1330422,131,8,1330422,Wokingham,1330434,Tier 2 (A rating)\nTier 2 (A rating),1330435,Intra Company Transfers (ICT)\nTier 2 General


In [11]:
df_name_sorted_imported['lowcase_name'] = df_name_sorted_imported['name'].str.lower()

In [12]:
df_name_sorted_imported[0:]

,style,name,left,top,width,height,city_top,city,tier_and_rating_top,tier_and_rating,sub_tier_top,sub_tier,lowcase_name
index,,,,,,,,,,,,,
0,position:absolute; border: textbox 1px solid; ...,@ Home Accommodation Services Ltd,30,266,160,8,266,London,278,Tier 2 (A rating)\nTier 5 (A rating),279,Tier 2 General\nCreative & Sporting,@ home accommodation services ltd
1,position:absolute; border: textbox 1px solid; ...,]performance s p a c e [,30,309,98,8,309,Folkestone,321,Tier 5 (A rating),322,Creative & Sporting,]performance s p a c e [
2,position:absolute; border: textbox 1px solid; ...,06472420,30,340,38,8,339,London,351,Tier 2 (A rating),353,Tier 2 General,06472420
3,position:absolute; border: textbox 1px solid; ...,0-two Maintenance,30,370,78,8,370,London,382,Tier 2 (A rating),383,Tier 2 General,0-two maintenance
4,position:absolute; border: textbox 1px solid; ...,1 Digitals Europe Limited,30,401,105,8,400,Slough,412,Tier 2 (A rating),414,Tier 2 General,1 digitals europe limited
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31292,position:absolute; border: textbox 1px solid; ...,Zytek Automotive Limited,30,1330349,106,8,1330348,Lichfield,1330360,Tier 2 (A rating)\nTier 2 (A rating),1330362,Tier 2 General\nIntra Company Transfers (ICT),zytek automotive limited
31293,position:absolute; border: textbox 1px solid; ...,Zytronic Displays Limited,30,1330392,106,8,1330391,Blaydon on Tyne,1330403,Tier 2 (A rating),1330405,Tier 2 General,zytronic displays limited
31294,position:absolute; border: textbox 1px solid; ...,ZyXEL Communications UK Ltd,30,1330422,131,8,1330422,Wokingham,1330434,Tier 2 (A rating)\nTier 2 (A rating),1330435,Intra Company Transfers (ICT)\nTier 2 General,zyxel communications uk ltd


In [10]:
dict_name_sorted_imported = df_name_sorted_imported.T.to_dict().values()

In [ ]:
import psycopg2

try:
    connection = psycopg2.connect(user="postgres",
                                  password="hidden",
                                  host="hidden",
                                  port="5432",
                                  database="uk-immigration")
    cursor = connection.cursor()
    processd = 0
    postgres_insert_query = """ INSERT INTO "find-job"."licensed-sponsors"("company-name", "city", "tier-and-rating", "sub-tier") VALUES (%s,%s,%s,%s)"""
    for record in dict_name_sorted_imported: 
        
        record_to_insert = (record['name'], record['city'], record['tier_and_rating'], record['sub_tier'])
        cursor.execute(postgres_insert_query, record_to_insert)

        connection.commit()
        processd = processd + 1
        
        if (processd % 10 == 0):
            print(processd)
       
    count = cursor.rowcount
    print (count, "Record inserted successfully into mobile table")

except (Exception, psycopg2.Error) as error :
    if(connection):
        print("Failed to insert record into mobile table", error)

finally:
    #closing database connection.
    if(connection):
#         cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [267]:


for record in dict_name_sorted_imported:
    print(record)
    print(record['name'])
    break
#     table.put_item(Item=student)


{'style': 'position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:30px; top:266px; width:160px; height:8px;', 'name': '@ Home Accommodation Services Ltd', 'left': 30, 'top': 266, 'width': 160, 'height': 8, 'city_top': 266, 'city': 'London', 'tier_and_rating_top': 278, 'tier_and_rating': 'Tier 2 (A rating)\nTier 5 (A rating)', 'sub_tier_top': 279, 'sub_tier': 'Tier 2 General\nCreative & Sporting'}
@ Home Accommodation Services Ltd


In [38]:
# df_name_sorted_imported['top'] = (df_name_sorted_imported['top'] / 2).apply(np.round) * 2

In [39]:
# df_name_sorted_imported

,style,name,left,top,width,height
index,,,,,,
0,position:absolute; border: textbox 1px solid; ...,@ Home Accommodation Services Ltd,30,266.0,160,8
1,position:absolute; border: textbox 1px solid; ...,]performance s p a c e [,30,308.0,98,8
2,position:absolute; border: textbox 1px solid; ...,06472420,30,340.0,38,8
3,position:absolute; border: textbox 1px solid; ...,0-two Maintenance,30,370.0,78,8
4,position:absolute; border: textbox 1px solid; ...,1 Digitals Europe Limited,30,400.0,105,8
...,...,...,...,...,...,...
31314,position:absolute; border: textbox 1px solid; ...,Zytek Automotive Limited,30,1330348.0,106,8
31315,position:absolute; border: textbox 1px solid; ...,Zytronic Displays Limited,30,1330392.0,106,8
31316,position:absolute; border: textbox 1px solid; ...,ZyXEL Communications UK Ltd,30,1330422.0,131,8


In [188]:
df_city_sorted_imported = pd.read_parquet('df_city_sorted.parquet.gzip')

In [189]:
df_city_sorted_imported

,style,name,left,top,width,height,name_top
index,,,,,,,
0,position:absolute; border: textbox 1px solid; ...,London,357,266,29,8,266
1,position:absolute; border: textbox 1px solid; ...,Folkestone,357,309,43,8,309
2,position:absolute; border: textbox 1px solid; ...,London,357,339,29,8,340
3,position:absolute; border: textbox 1px solid; ...,London,357,370,29,8,370
4,position:absolute; border: textbox 1px solid; ...,Slough,357,400,27,8,401
...,...,...,...,...,...,...,...
31275,position:absolute; border: textbox 1px solid; ...,Lichfield,357,1330348,32,8,1330349
31276,position:absolute; border: textbox 1px solid; ...,Blaydon on Tyne,357,1330391,67,8,1330392
31277,position:absolute; border: textbox 1px solid; ...,Wokingham,357,1330422,47,8,1330422


In [42]:
# df_city_sorted_imported['top'] = (df_city_sorted_imported['top'] / 2).apply(np.round) * 2

In [43]:
# df_city_sorted_imported

,style,name,left,top,width,height
index,,,,,,
0,position:absolute; border: textbox 1px solid; ...,London,357,266.0,29,8
1,position:absolute; border: textbox 1px solid; ...,Folkestone,357,308.0,43,8
2,position:absolute; border: textbox 1px solid; ...,London,357,340.0,29,8
3,position:absolute; border: textbox 1px solid; ...,London,357,370.0,29,8
4,position:absolute; border: textbox 1px solid; ...,Slough,357,400.0,27,8
...,...,...,...,...,...,...
31275,position:absolute; border: textbox 1px solid; ...,Lichfield,357,1330348.0,32,8
31276,position:absolute; border: textbox 1px solid; ...,Blaydon on Tyne,357,1330392.0,67,8
31277,position:absolute; border: textbox 1px solid; ...,Wokingham,357,1330422.0,47,8


In [50]:
df_tier_and_rating_sorted_imported = pd.read_parquet('df_tier_and_rating_sorted.parquet.gzip')

In [51]:
df_tier_and_rating_sorted_imported

,style,name,left,top,width,height
index,,,,,,
0,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating)\nTier 5 (A rating),582,278,62,21
1,position:absolute; border: textbox 1px solid; ...,Tier 5 (A rating),582,321,62,8
2,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,351,62,8
3,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,382,62,8
4,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,412,62,8
...,...,...,...,...,...,...
32673,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating)\nTier 2 (A rating),582,1330360,62,21
32674,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating),582,1330403,62,8
32675,position:absolute; border: textbox 1px solid; ...,Tier 2 (A rating)\nTier 2 (A rating),582,1330434,62,21


In [ ]:
df_tier_and_rating_sorted_imported['top'] = df_tier_and_rating_sorted_imported['top']/5

In [13]:
df_sub_tier_sorted_imported = pd.read_parquet('df_sub_tier_sorted.parquet.gzip')

In [19]:
df_sub_tier_sorted_imported['top'] = (df_sub_tier_sorted_imported['top'] / 3).apply(np.ceil) * 3

In [20]:
df_sub_tier_sorted_imported

,style,name,left,top,width,height
index,,,,,,
0,position:absolute; border: textbox 1px solid; ...,Tier 2 General\nCreative & Sporting,684,279.0,77,21
1,position:absolute; border: textbox 1px solid; ...,Creative & Sporting,684,324.0,77,8
2,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,354.0,57,8
3,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,384.0,57,8
4,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,414.0,57,8
...,...,...,...,...,...,...
32583,position:absolute; border: textbox 1px solid; ...,Tier 2 General\nIntra Company Transfers (ICT),684,1330362.0,121,21
32584,position:absolute; border: textbox 1px solid; ...,Tier 2 General,684,1330407.0,57,8
32585,position:absolute; border: textbox 1px solid; ...,Intra Company Transfers (ICT)\nTier 2 General,684,1330437.0,121,21


In [127]:
df_newdata = df_name_sorted_imported

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.numeric.Int64Index'> with these indexers [city_name] of <class 'str'>

In [113]:
df_newdata['city_top'] = df_city_sorted_imported['top']
df_newdata['city_name'] = df_city_sorted_imported['name']
df_newdata['city_index'] = df_city_sorted_imported['index']

KeyError: 'index'

In [78]:
df_newdata

,style,name,left,top,width,height,city_top,diff,city_name
index,,,,,,,,,
0,position:absolute; border: textbox 1px solid; ...,@ Home Accommodation Services Ltd,30,266,160,8,266.0,0.0,London
1,position:absolute; border: textbox 1px solid; ...,]performance s p a c e [,30,309,98,8,309.0,0.0,Folkestone
2,position:absolute; border: textbox 1px solid; ...,06472420,30,340,38,8,339.0,1.0,London
3,position:absolute; border: textbox 1px solid; ...,0-two Maintenance,30,370,78,8,370.0,0.0,London
4,position:absolute; border: textbox 1px solid; ...,1 Digitals Europe Limited,30,401,105,8,400.0,1.0,Slough
...,...,...,...,...,...,...,...,...,...
31314,position:absolute; border: textbox 1px solid; ...,Zytek Automotive Limited,30,1330349,106,8,NaN,NaN,NaN
31315,position:absolute; border: textbox 1px solid; ...,Zytronic Displays Limited,30,1330392,106,8,NaN,NaN,NaN
31316,position:absolute; border: textbox 1px solid; ...,ZyXEL Communications UK Ltd,30,1330422,131,8,NaN,NaN,NaN


In [79]:
df_newdata['diff'] = df_newdata['top'] - df_newdata['city_top']

In [80]:
df_newdata['diff'] = df_newdata['diff'].abs()

In [81]:
df_newdata['diff']

index
0        0.0
1        0.0
2        1.0
3        0.0
4        1.0
        ... 
31314    NaN
31315    NaN
31316    NaN
31322    NaN
31323    NaN
Name: diff, Length: 31324, dtype: float64

In [130]:
df_newdata_big_diff = df_newdata[df_newdata['diff'] > 1 ]

In [131]:
df_newdata.shape

(31324, 9)

In [132]:
df_newdata_big_diff.shape

(30294, 9)

In [133]:
df_newdata_big_diff

,style,name,left,top,width,height,city_top,diff,city_name
index,,,,,,,,,
28,position:absolute; border: textbox 1px solid; ...,16HarleyCo Limited,30,1378,81,8,1408.0,30.0,Jewellery quarter
29,position:absolute; border: textbox 1px solid; ...,16printing Ltd,30,1409,58,8,1439.0,30.0,London
30,position:absolute; border: textbox 1px solid; ...,17 Capital Services Ltd,30,1439,95,8,1469.0,30.0,London
31,position:absolute; border: textbox 1px solid; ...,18 Feet & Rising Limited,30,1470,101,8,1500.0,30.0,London
32,position:absolute; border: textbox 1px solid; ...,1854 Media Limited,30,1500,80,8,1531.0,31.0,London
...,...,...,...,...,...,...,...,...,...
31274,position:absolute; border: textbox 1px solid; ...,Zion Capital Limited,30,1328466,83,8,1330318.0,1852.0,Hounslow
31278,position:absolute; border: textbox 1px solid; ...,Zion Christian Church,30,1328496,91,8,1330465.0,1969.0,London
31279,position:absolute; border: textbox 1px solid; ...,Zipcube ltd,30,1328539,46,8,1330495.0,1956.0,London


In [134]:
df_newdata_big_diff.index

Int64Index([   28,    29,    30,    31,    32,    27,    35,    36,    37,
               38,
            ...
            31270, 31271, 31272, 31277, 31273, 31274, 31278, 31279, 31275,
            31276],
           dtype='int64', name='index', length=30294)

In [135]:
df_newdata_big_diff.index[0]

28

In [136]:
df_newdata['top']

index
0            266
1            309
2            340
3            370
4            401
          ...   
31314    1330349
31315    1330392
31316    1330422
31322    1330465
31323    1330496
Name: top, Length: 31324, dtype: int64